In [7]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform


def cal_lap_score(features, D, L):
    features_ = features - np.sum((features @ D) / np.sum(D))
    L_score = (features_ @ L @ features_) / (features_ @ D @ features_)
    return L_score


def get_k_nearest(dist, k, sample_index):
    return sorted(
        range(len(dist)),
        key=lambda i: dist[i] if i != sample_index else np.inf
    )[:k] + [sample_index]


def laplacian_score(df_arr, label=None, **kwargs):
    kwargs.setdefault("k_nearest", 5)
    distances = pdist(df_arr, metric='euclidean')
    dist_matrix = squareform(distances)
    edge_sparse_matrix = pd.DataFrame(
        np.zeros((df_arr.shape[0], df_arr.shape[0])),
        dtype=int
    )
    if label is None:
        for row_id, row in enumerate(dist_matrix):
            k_nearest_id = get_k_nearest(row, kwargs["k_nearest"], row_id)
            edge_sparse_matrix.iloc[k_nearest_id, k_nearest_id] = 1
    else:
        label = np.array(label)
        unique_label = np.unique(label)
        for i in unique_label:
            group_index = np.where(label == i)[0]
            edge_sparse_matrix.iloc[group_index, group_index] = 1
    S = dist_matrix * edge_sparse_matrix
    del dist_matrix, edge_sparse_matrix
    D = np.diag(S.sum(axis=1))
    L = D - S
    del S
    features_lap_score = np.apply_along_axis(
        func1d=lambda f: cal_lap_score(f, D, L), axis=0, arr=df_arr
    )
    return features_lap_score

In [8]:
file_path = '../data/URIELPlus_Union.csv'

df = pd.read_csv(file_path, index_col=0)
laplacian_scores = laplacian_score(df)

In [9]:
for n_features in range(500, 701, 100):
    feature_indices = np.argsort(laplacian_scores)[::-1][:n_features]
    filtered_data = df.iloc[:, feature_indices]
    # save to csv
    filtered_data.to_csv(f'../selection_result/base_laplacian_{n_features}.csv')